In [3]:
import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# CSVファイルを読み込む
try:
    df = pd.read_csv('/Users/dangararara/lecture/miraisouzou/csv/odaiba_reviews.csv')
except FileNotFoundError:
    print("エラー: 'odaiba_reviews.csv'が見つかりません。")
    df = pd.DataFrame()

# JanomeのTokenizerを準備
t = Tokenizer()

# ストップワード
stop_words = [
    'こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', '的', '的', '気', '感',
    'いる', 'する', 'なる', 'ある', 'ない', 'くる', 'いく', '思う', '言う',
    '美味しい', 'うまい', '普通', '良い', '悪い', '最高', '本当', '今回', '前回', 
    '初', '再訪', '人気', '有名', '定番', '満足', '好き', 'おすすめ', 'ごちそうさま', '様',
    '料理', '食事', 'メニュー', 'ドリンク', 'デザート', '一品', 'ご飯', 'ビール',  '肉', '魚', '野菜',
    '食べる', '飲む', '行く', '来る', '入る', '出る', '頼む', '注文', 
    '訪問', '利用', '頂く', 'いただく', '会計',
    '時間', '日', '時', '分', 
    'お店', 'レストラン', 'こちら', 'ところ', '場所', '雰囲気', '店内', 
    '外観', '席', 'スタッフ', '店員', 
    '自分', '私', '人',
    '一つ', 'たくさん', '少し', 'ちょっと', 'かなり', '非常', 'いろいろ', '色々', 
    '種類', '豊富', '一番', '円', '...',
]


# <--- 変更点：Tri-gram用のTokenizer関数 ---
def tokenize_for_trigrams(text):
    """テキストを形態素解析し、Tri-gramの文字列を返す"""
    # まずは単語に分解
    tokens = []
    for token in t.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    
    # 単語リストからTri-gramを作成
    trigrams = [f"{tokens[i]}_{tokens[i+1]}_{tokens[i+2]}" for i in range(len(tokens) - 2)]
    return ' '.join(trigrams)

if not df.empty:
    shop_docs = df.groupby('shop_name')['review_text'].apply(lambda x: ' '.join(x.dropna())).reset_index()
    
    print("各店舗の口コミをTri-gramに変換中...")
    # <--- 変更点：Tri-gram用の関数を適用 ---
    shop_docs['tokens'] = shop_docs['review_text'].apply(tokenize_for_trigrams)
    
    corpus = shop_docs['tokens'].tolist()
    shop_names = shop_docs['shop_name'].tolist()

    if corpus:
        print("Tri-gramのTF-IDFスコアを計算中...")
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)
        tfidf_matrix = vectorizer.fit_transform(corpus)
        feature_names = vectorizer.get_feature_names_out()

        print("\n--- 店舗ごとの特徴語（Tri-gramスコア上位）---\n")
        
        for i, shop_name in enumerate(shop_names):
            print(f"【{shop_name}】")
            row = tfidf_matrix.toarray()[i]
            top_n_indices = row.argsort()[::-1][:7]
            
            for index in top_n_indices:
                if row[index] > 0:
                    # <--- 変更点：見やすいようにアンダースコアを置換 ---
                    display_name = feature_names[index].replace('_', ', ')
                    print(f"  - {display_name} (スコア: {row[index]:.3f})")
            print("-" * 20)

各店舗の口コミをTri-gramに変換中...
Tri-gramのTF-IDFスコアを計算中...

--- 店舗ごとの特徴語（Tri-gramスコア上位）---

【1129 by Ogawa】
  - デックス, 東京, ビーチ (スコア: 0.286)
  - 東京, ビーチ, 6 (スコア: 0.279)
  - 台場, 海浜, 公園 (スコア: 0.194)
  - ビーチ, 6, 階 (スコア: 0.168)
  - ハンバーグ, 200, g (スコア: 0.140)
  - 海浜, 公園, 駅 (スコア: 0.126)
  - 牛, タン, カレー (スコア: 0.107)
--------------------
【82 築地店】
  - 築地, 駅, 近く (スコア: 0.197)
  - メトロ, 日比谷線, 築地 (スコア: 0.186)
  - 東京, メトロ, 日比谷線 (スコア: 0.183)
  - 日比谷線, 築地, 駅 (スコア: 0.177)
  - 東, 銀座, 店 (スコア: 0.152)
  - 00,  (スコア: 0.148)
  - 一, 入る, やすい (スコア: 0.124)
--------------------
【ABURI 百貫 有明ガーデン店】
  - 回転, 寿司, 店 (スコア: 0.255)
  - ガーデン, 回転, 寿司 (スコア: 0.200)
  - 回転, 寿司, 屋 (スコア: 0.153)
  - 有明, ガーデン, 内 (スコア: 0.136)
  - 有明, ガーデン, 店 (スコア: 0.135)
  - グルメ, 回転, 寿司 (スコア: 0.100)
  - 有明, ガーデン, 寿司 (スコア: 0.100)
--------------------
【ARIAKE ARENA CAFE】
  - 有明, アリーナ, 行く (スコア: 0.274)
  - ライブ, 有明, アリーナ (スコア: 0.261)
  - 有明, アリーナ, イベント (スコア: 0.252)
  - , 2, 日間 (スコア: 0.137)
  - スチュワート, ライブ (スコア: 0.137)
  - 暑い, さ, へとへと (スコア: 0.137)
  - 書く, せる, いただく (ス